<a href="https://colab.research.google.com/github/fabiopauli/Qwen3.5-colab/blob/main/LlamacppWebServer_Qwen3_5_Cloudflare_tunnel_public_IP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt-get update -qq
!apt-get install -y build-essential cmake git -qq
!pip install -q huggingface_hub[cli]

# Build latest llama.cpp with full CUDA support for L4
!git clone --depth 1 https://github.com/ggml-org/llama.cpp.git
%cd llama.cpp
!cmake -B build -DGGML_CUDA=ON -DCMAKE_BUILD_TYPE=Release
!cmake --build build --config Release -j4

print("✅ Build finished! Ready for Cell 2.")
%cd /content

In [13]:

import subprocess, time, threading, os
from IPython.display import clear_output

clear_output()

# Set your custom port here
PORT = 8081  # Change this if needed

# Unsloth recommended cache (forces clean download to correct folder)
os.environ["LLAMA_CACHE"] = "unsloth/Qwen3.5-27B-GGUF"

print("📥 First run will auto-download UD-Q4_K_XL.gguf (~16.7 GB from Unsloth)...")

# === Install lsof if needed & Kill any process on the port ===
!apt-get install -y lsof  # Usually already installed, but to be safe
print(f"🛑 Killing any process on port {PORT}...")
!kill $(lsof -t -i:{PORT}) 2>/dev/null || true

# === Cloudflared Tunnel ===
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb 2>/dev/null || true

def start_tunnel():
    tunnel_process = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{PORT}", "--no-autoupdate"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    for line in iter(tunnel_process.stdout.readline, ''):
        print(line.strip())
    tunnel_process.stdout.close()
    tunnel_process.wait()

print("🌐 Starting free public Cloudflare Tunnel...")
tunnel_thread = threading.Thread(target=start_tunnel, daemon=True)
tunnel_thread.start()
time.sleep(8)

# === llama-server + WebUI with YOUR exact settings ===
print("\n🚀 Starting llama.cpp server + built-in WebUI")
print("   Model : unsloth/Qwen3.5-27B UD-Q4_K_XL (16.7 GB)")
print("   Context: 16384 | Thinking: DISABLED | Flash Attention ON")
print("   Temp 0.7 | top-p 0.8 | top-k 20 | min-p 0.00")
print("   Wait 45–120 seconds while model loads on L4...\n")
print("═"*85)
print(f"✅ YOUR PUBLIC WEBUI LINK will appear below (something.trycloudflare.com on port {PORT})")
print("   Open it in any browser → full ChatGPT-style UI")
print("═"*85)

server_cmd = f"""
/content/llama.cpp/build/bin/llama-server \
  -hf unsloth/Qwen3.5-27B-GGUF:UD-Q4_K_XL \
  --jinja \
  -c 16384 \
  --n-gpu-layers 99 \
  --temp 0.7 \
  --top-p 0.8 \
  --top-k 20 \
  --min-p 0.00 \
  --chat-template-kwargs '{{"enable_thinking": false}}' \
  --host 0.0.0.0 \
  --port {PORT} \
  --alias "Qwen3.5-27B-Unsloth-UD-Q4"
"""

!{server_cmd}

📥 First run will auto-download UD-Q4_K_XL.gguf (~16.7 GB from Unsloth)...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lsof is already the newest version (4.93.2+dfsg-1.1build2).
0 upgraded, 0 newly installed, 0 to remove and 86 not upgraded.
🛑 Killing any process on port 8081...
(Reading database ... 121856 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2026.2.0) over (2026.2.0) ...
Setting up cloudflared (2026.2.0) ...
Processing triggers for man-db (2.10.2-1) ...
🌐 Starting free public Cloudflare Tunnel...
2026-02-26T12:19:31Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the rig